In [1]:
from os.path import join
import pandas as pd
import torch
from tqdm.auto import tqdm

path = join("..", "data", "slowosiec_all_data.json.gz")

df = pd.read_json(path)
df.head()

,"('emotions',)","('markedness',)","('text',)","('lemma',)","('org_emotion',)"
0,smutek,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie,smutek
1,zlosc,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie,złość
10,oczekiwanie,0.5,Kibice kieleckiego beniaminka przyszli na pier...,beniaminek,cieszenie się na coś oczekiwanego
100,neutralny,0.5,Oddział najlepszych strzelców stanowił osobist...,gwardia,
1000,smutek,-0.5,"Moja praca jest beznadziejna, nic się w niej n...",dupogodzina,smutek


In [2]:
from transformers import XLMTokenizer, RobertaModel

tokenizer = XLMTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

In [3]:
model = model.to("cuda")

embeddings = []

for emotion, text in tqdm(zip(df["('emotions',)"], df["('text',)"])):
    encoded_input = tokenizer.encode(text, return_tensors='pt')
    encoded_input = encoded_input.to("cuda")
    with torch.no_grad():
        outputs = model(encoded_input)
    embeddings.append((outputs[1].detach().cpu().numpy(), emotion))

In [6]:
import pickle
save_path = join("..", "data", "slowosiec_all_data_embeddings.pickle")

with open(save_path, 'wb') as handle:
    pickle.dump((embeddings), handle)